In [99]:
import pandas as pd

In [49]:
# Load datasets
df1 = pd.read_csv('202305-citibike-tripdata_1.csv')
df2 = pd.read_csv('202305-citibike-tripdata_2.csv')

# Convert timestamps to datetime objects
df1['timestamp'] = pd.to_datetime(df1['started_at'])
df2['timestamp'] = pd.to_datetime(df2['started_at'])

# Subset and rename columns
df1 = df1[['timestamp','start_lat', 'end_lat']]
df1.columns = ['timestamp','latitude', 'longitude']

df2 = df2[['timestamp','start_lat', 'end_lat']]
df2.columns = ['timestamp','lat', 'lon']

<ipython-input-49-9980d2db21fa>:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('202305-citibike-tripdata_1.csv')
<ipython-input-49-9980d2db21fa>:3: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('202305-citibike-tripdata_2.csv')


In [50]:
# Sort datasets by timestamp
df1 = df1.sort_values(by='timestamp')
df2 = df2.sort_values(by='timestamp')

# Drop na
df1 = df1.dropna(subset=['latitude', 'longitude'])
df2 = df2.dropna(subset=['lat', 'lon'])

In [86]:
import pandas as pd
from shapely.geometry import Point

# Create Shapely Point geometries from latitude and longitude
df1['geometry'] = [Point(lon, lat) for lon, lat in zip(df1['longitude'], df1['latitude'])]
df2['geometry'] = [Point(lon, lat) for lon, lat in zip(df2['lon'], df2['lat'])]

# Buffer the points (e.g., with a radius of 1 degree)
buffer_radius = .001  # Modify this value as needed
df1['buffer'] = df1['geometry'].apply(lambda point: point.buffer(buffer_radius))

/Users/avichudasama/opt/anaconda3/envs/sandbox/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [89]:
df1 = df1.reset_index().rename(columns={'index': 'df1index'})

df1.head(2)

,df1index,timestamp,latitude,longitude,geometry,buffer
0,531396,2023-05-01 00:00:24,40.735081,40.758057,POINT (40.758057 40.735080957),"POLYGON ((40.759057 40.735080957, 40.759052184..."
1,800632,2023-05-01 00:00:28,40.774925,40.759345,POINT (40.75934501 40.77492513),"POLYGON ((40.760345009999995 40.77492513, 40.7..."


In [90]:
df2 = df2.reset_index().rename(columns={'index': 'df2index'})

df2.head(2)

,df2index,timestamp,lat,lon,geometry
0,62582,2023-05-01 00:00:03,40.680467,40.669920,POINT (40.66992 40.680466652)
1,840640,2023-05-01 00:00:13,40.746647,40.737169,POINT (40.73716919369882 40.746647)


In [91]:
# merge dataframes
common_timestamps = pd.merge_asof(df1, df2, on='timestamp', tolerance=pd.Timedelta('2 minute'))
common_timestamps.head(2)

,df1index,timestamp,latitude,longitude,geometry_x,buffer,df2index,lat,lon,geometry_y
0,531396,2023-05-01 00:00:24,40.735081,40.758057,POINT (40.758057 40.735080957),"POLYGON ((40.759057 40.735080957, 40.759052184...",282853.0,40.835144,40.865570,POINT (40.86557 40.835143685)
1,800632,2023-05-01 00:00:28,40.774925,40.759345,POINT (40.75934501 40.77492513),"POLYGON ((40.760345009999995 40.77492513, 40.7...",253909.0,40.685327,40.687881,POINT (40.687880939612775 40.685327172)


pd.merge_asof() is a function provided by the Pandas library in Python, used for merging two datasets based on nearest key columns. It's particularly useful when dealing with time-series data or datasets with ordered keys.

The basic idea of merge_asof() is to merge two datasets on the nearest key column. It's similar to a left-join operation, but instead of matching exactly on the key column, it matches on the nearest key that is less than or equal to the current key from the left dataframe.

In [92]:
# Compare spatial coordinates
# compare timestamps to identify common time intervals between the two datasets.

distances = []
for idx, row in common_timestamps.iterrows():
    coord1 = (row['lat'], row['lon'])
    buffered_pt = row['buffer']
    
    coord2 = (row['latitude'], row['longitude'])
    point = row['geometry_y']

    try:
        common_timestamps.at[idx, 'within'] = point.within(buffered_pt)
        common_timestamps.at[idx, 'intersects'] = point.intersects(buffered_pt)

    except:
        pass

from geopy.distance import great_circle
    try:
        distance = great_circle(coord1, coord2).miles
        common_timestamps.at[idx, 'delta'] = distance
        distances.append(distance)
        # print(f"Distance between points at {row['timestamp']}: {distance} miles")
    except:
        print(idx, coord1, coord2)
        common_timestamps.at[idx, 'delta'] = 'N/A'

great_circle is a function provided by the geopy.distance module in Python, which calculates the great-circle distance between two points on the Earth's surface.

A great circle is the shortest path between two points on a sphere, which corresponds to the shortest path along the surface of the Earth when dealing with geographical coordinates (latitude and longitude).

Here's a basic example of how to use great_circle:

```
from geopy.distance import great_circle
    try:
        distance = great_circle(coord1, coord2).miles
        common_timestamps.at[idx, 'delta'] = distance
        distances.append(distance)
        # print(f"Distance between points at {row['timestamp']}: {distance} miles")
    except:
        print(idx, coord1, coord2)
        common_timestamps.at[idx, 'delta'] = 'N/A'
```

In [95]:
common_timestamps['within'].value_counts()

False    995430
True        904
Name: within, dtype: int64

In [96]:
common_timestamps['intersects'].value_counts()

False    995430
True        904
Name: intersects, dtype: int64

In [98]:
common_timestamps.head(2)

,df1index,timestamp,latitude,longitude,geometry_x,buffer,df2index,lat,lon,geometry_y,within,intersects
0,531396,2023-05-01 00:00:24,40.735081,40.758057,POINT (40.758057 40.735080957),"POLYGON ((40.759057 40.735080957, 40.759052184...",282853.0,40.835144,40.865570,POINT (40.86557 40.835143685),False,False
1,800632,2023-05-01 00:00:28,40.774925,40.759345,POINT (40.75934501 40.77492513),"POLYGON ((40.760345009999995 40.77492513, 40.7...",253909.0,40.685327,40.687881,POINT (40.687880939612775 40.685327172),False,False
